In [1]:
from igv import IGV, Reference, Track
import pandas as pd

In [2]:
import sys, time
sys.path.append("/users/pshannon/github/fimoService/client-python")
from FimoClient import *
sys.path.append("/users/pshannon/github/getDNAService/client-python")
from GetDNAClient import *

In [3]:
fimo = FimoClient("whovian", 5558)
assert(fimo.getHost() == 'whovian:5558')
dnaService = GetDNAClient("hg38")
assert(dnaService.getSequence("chr1", 1, 5) == 'NNNNN')

## The VRK2-associated [SNP](https://www.ncbi.nlm.nih.gov/projects/SNP/snp_ref.cgi?searchType=adhoc_search&type=rs&rs=rs13384219)

hg38:  chr2:57907323

In [28]:
chrom = "chr2"
loc = 57907323
shoulder = 7
snpLocus = "%s:%d-%d" % (chrom, loc-shoulder, loc+shoulder)
viewRange = "chr2:57,907,026-57,907,985"

In [29]:
igv = IGV(locus=viewRange, reference=Reference(id="hg38"), 
          tracks=[Track(
                 name="Genes", 
                 url="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz",
                 indexURL="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz.tbi",
                 display_mode="EXPANDED")])
igv

In [30]:
trackTbl = pd.DataFrame([[chrom, loc-shoulder, loc+shoulder, "snp"]])
trackTbl.to_csv("vrk2Snp.bed", sep="\t", header=False, index=False)
newTrack = Track(name="rs13384219 ", format="bed", indexed=False, 
                 url="http://whovian:10005/files/shared/vrk2Snp.bed", 
                 display_mode='EXPANDED');

In [31]:
igv.load_track(newTrack)

Loading track into IGV.js


In [33]:
snpLocus

'chr2:57907316-57907330'

In [34]:
baseSequence = dnaService.getSequence(chrom, loc-shoulder, loc+shoulder)
print(baseSequence)

AGCATGCAAATTAGA


In [35]:
end1 = shoulder
start2 = shoulder + 1
end2 = len(baseSequence)
mutA = baseSequence[0:end1] + 'A' + baseSequence[start2:end2]
mutG = baseSequence[0:end1] + 'G' + baseSequence[start2:end2]
mutT = baseSequence[0:end1] + 'T' + baseSequence[start2:end2]

In [36]:
pd.DataFrame([[baseSequence], [mutG]])

,0
0,AGCATGCAAATTAGA
1,AGCATGCGAATTAGA


In [37]:
tbl_fimo = fimo.request({"wt": baseSequence, "mutG": mutG})

the highest-scoring motif on the + strand [MA0784.1](http://jaspar.genereg.net/cgi-bin/jaspar_db.pl?rm=present&collection=CORE&ID=MA0784.1)

In [38]:
tbl_fimo

,#pattern name,matched sequence,p-value,q-value,score,sequence name,start,stop,strand
0,MA0507.1,CTAATTTGCATGC,0.000005,0.000062,15.3793,wt,2,14,-
1,MA0784.1,AGCATGCAAATTAG,0.000008,0.000062,14.0862,wt,1,14,+
2,MA0785.1,AGCATGCAAATT,0.000015,0.000242,13.0600,wt,1,12,+
3,MA0792.1,CATGCAAAT,0.000024,0.000676,12.3186,wt,3,11,+
4,MA0788.1,AGCATGCAAATTA,0.000025,0.000301,12.4400,wt,1,13,+
5,MA0787.1,GCATGCAAATTA,0.000033,0.000527,12.0323,wt,2,13,+
6,MA0068.2,CGAATTAG,0.000040,0.001270,11.4179,mutG,7,14,+
7,MA0786.1,GCATGCAAATTA,0.000051,0.000821,11.5181,wt,2,13,+
8,MA0789.1,CATGCAAAT,0.000079,0.002220,10.8571,wt,3,11,+
9,MA0068.2,CTAATTCG,0.000084,0.001340,10.9851,mutG,7,14,-


In [16]:
import psycopg2 as psql

In [21]:
db  = psql.connect("dbname=hg38 user=pshannon")

In [27]:
pd.read_sql_query("select * from motifsgenes where motif in ('MA0068.2')", db)

,motif,tf
0,MA0068.2,PAX4
1,MA0068.2,PAX6
2,MA0068.2,PAX3
3,MA0068.2,PAX7


In [39]:
pd.read_sql_query("select * from motifsgenes where motif in ('MA0507.1', 'MA0784.1', 'MA0785.1')", db)

,motif,tf
0,MA0507.1,POU2F2
1,MA0784.1,POU1F1
2,MA0785.1,POU2F1
3,MA0507.1,HDX
4,MA0507.1,HMBOX1
5,MA0507.1,POU6F1
6,MA0507.1,POU6F2
7,MA0507.1,POU5F1
8,MA0507.1,POU5F2
9,MA0507.1,POU4F1
